In [11]:


from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.memory import ConversationSummaryMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(
    llm=llm
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
# chat = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])



In [12]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [13]:
get_history()

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds by expressing admiration for his location and expresses a desire to visit.'}